In [3]:
# pip install scikit-learn==1.5.1

In [2]:
import pandas as pd
import json
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

salaire = pd.read_csv("data/net_salary_per_town_categories.csv")
# Pré-traitement des données salaire
new_column_names_salaire = {
    'SNHM14': 'salaire',
    'SNHMC14': 'salaire_cadre',
    'SNHMP14': 'salaire_cadre_moyen',
    'SNHME14': 'salaire_employe',
    'SNHMO14': 'salaire_travailleur',
    'SNHMF14': 'salaire_femme',
    'SNHMFC14': 'salaire_cadre_femme',
    'SNHMFP14': 'salaire_cadre_moyen_femme',
    'SNHMFE14': 'salaire_employe_femme',
    'SNHMFO14': 'salaire_travailleur_femme',
    'SNHMH14': 'salaire_homme',
    'SNHMHC14': 'salaire_cadre_homme',
    'SNHMHP14': 'salaire_cadre_moyen_homme',
    'SNHMHE14': 'salaire_employe_homme',
    'SNHMHO14': 'salaire_travailleur_homme',
    'SNHM1814': 'salaire_18-25',
    'SNHM2614': 'salaire_26-50',
    'SNHM5014': 'salaire_+50',
    'SNHMF1814': 'salaire_18-25_femme',
    'SNHMF2614': 'salaire_26-50_femme',
    'SNHMF5014': 'salaire_+50_femme',
    'SNHMH1814': 'salaire_18-25_homme',
    'SNHMH2614': 'salaire_26-50_homme',
    'SNHMH5014': 'salaire_+50_homme'
}

salaire = salaire.rename(columns=new_column_names_salaire)

salaire = salaire.drop(['CODGEO', 'LIBGEO'],axis=1)
salaire_FA = salaire.drop(['salaire_femme','salaire_26-50','salaire_26-50_femme','salaire_26-50_homme'],axis=1)
# salaire_FA_cible=salaire_FA['salaire']
# salaire_FA_feats = salaire_FA.drop(['salaire'],axis=1)

salaire_box = salaire
salaire_box['salaire'] = salaire_box['salaire'].astype(str)


# Sélectionner toutes les colonnes numériques sauf la variable cible
variables_numeriques = salaire_box.drop(['salaire'],axis=1)

# Boucle sur chaque colonne pour la discrétisation
for col in variables_numeriques:
    # Discrétisation en 5 intervalles
    salaire_box[f'{col}_discretise'] = pd.cut(salaire_box[col], bins=5)
    # Convertir la variable discrétisée en type catégoriel
    salaire_box[f'{col}_discretise'] = salaire_box[f'{col}_discretise'].astype(CategoricalDtype(ordered=True))


# Sélectionner les colonnes discrétisées
colonnes_discretisees = [col for col in salaire_box.columns if '_discretise' in col]

# Extraire uniquement les colonnes sélectionnées
salaire_disc = salaire_box[colonnes_discretisees]

salaire_disc.head()

label_pos = salaire_FA.columns[25:]
salaire0 = salaire_FA.drop(label_pos, axis=1)

cible = salaire_box['salaire']
table = pd.concat([salaire_disc,cible],axis=1)


# Encodage des variables catégorielles
encoder = LabelEncoder()

columns = table.columns

for column in columns:
    if column != "salaire":
        table[column] = encoder.fit_transform(table[column])

        
# Drop des variables trop corrélées avec la variable cible ( > 0.9) et autres
table = table.drop(['salaire_femme_discretise','salaire_26-50_discretise','salaire_26-50_femme_discretise','salaire_26-50_homme_discretise','salaire_cadre_moyen_discretise','salaire_travailleur_discretise',
                'salaire_cadre_femme_discretise','salaire_cadre_moyen_femme_discretise','salaire_employe_femme_discretise', 'salaire_travailleur_femme_discretise',
                 'salaire_cadre_homme_discretise', 'salaire_cadre_moyen_homme_discretise', 'salaire_employe_homme_discretise', 
                'salaire_travailleur_homme_discretise',	'salaire_18-25_discretise','salaire_18-25_femme_discretise',
                'salaire_18-25_homme_discretise','salaire_+50_homme_discretise'],axis=1)


# Séparation de la variable cible/features
feats=table.drop(['salaire'],axis=1)
target=table['salaire']


# Création d'un jeu d'entrainement et de test
X_train2, X_test2, y_train2, y_test2 = train_test_split(feats, target, random_state = 42)


# Entrainement du modèle
model = RandomForestRegressor(verbose = 1, random_state=42,max_depth=9)
model.fit(X_train2, y_train2)


import pickle

# Sauvegarder le modèle dans un fichier
with open('modele.pkl', 'wb') as fichier_modele:
    pickle.dump(model, fichier_modele)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [5]:
X_train2

,salaire_cadre_discretise,salaire_employe_discretise,salaire_homme_discretise,salaire_+50_discretise,salaire_+50_femme_discretise
789,0,0,0,0,0
3706,1,0,0,0,0
4564,0,0,0,0,0
2099,1,0,0,0,0
871,0,1,0,0,0
...,...,...,...,...,...
4426,0,0,0,0,0
466,0,0,0,0,0
3092,0,0,0,0,0
3772,0,0,0,0,0


In [6]:
# Calculer les statistiques descriptives
description = feats.describe()

# Obtenir les valeurs min et max de chaque feature
min_max_dict = {
    feature: {'min': description.at['min', feature], 'max': description.at['max', feature]}
    for feature in description.columns
}

# Enregistrer dans un fichier JSON
with open('feature_min_max.json', 'w') as json_file:
    json.dump(min_max_dict, json_file, indent=4)

In [7]:

# Encoder la target
encoded_target, unique_values = pd.factorize(table['salaire'])

# Créer un mapping de l'encodage
target_mapping = {index: value for index, value in enumerate(unique_values)}

# Enregistrer le mapping dans un fichier JSON
with open('target_encoding.json', 'w') as json_file:
    json.dump(target_mapping, json_file, indent=4)

In [8]:
table.head(60)

,salaire_cadre_discretise,salaire_employe_discretise,salaire_homme_discretise,salaire_+50_discretise,salaire_+50_femme_discretise,salaire
0,1,0,0,0,0,13.7
1,0,1,0,0,0,13.5
2,1,1,0,0,0,13.5
3,0,1,0,0,0,12.9
4,0,1,0,0,0,13.0
5,0,1,0,0,0,13.9
6,1,1,0,0,0,12.4
7,0,1,0,0,0,14.0
8,0,0,0,0,0,11.5
9,1,0,0,0,0,12.4


In [9]:
# 1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,2,0,0,0,13.7

In [10]:
#1,0,1,0,2,1,1,1,1,2,0,1,0,0,1,2,1,0,1,18.0 (43)

In [11]:
data_pred = {
        'Variables': ['salaire_cadre_discretise','salaire_cadre_moyen_discretise','salaire_employe_discretise',	'salaire_travailleur_discretise',
                'salaire_cadre_femme_discretise','salaire_cadre_moyen_femme_discretise','salaire_employe_femme_discretise', 'salaire_travailleur_femme_discretise',
                'salaire_homme_discretise', 'salaire_cadre_homme_discretise', 'salaire_cadre_moyen_homme_discretise', 'salaire_employe_homme_discretise', 
                'salaire_travailleur_homme_discretise',	'salaire_18-25_discretise','salaire_+50_discretise','salaire_18-25_femme_discretise','salaire_+50_femme_discretise',
                'salaire_18-25_homme_discretise','salaire_+50_homme_discretise','salaire'],
        'Prédiction N°1': [1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,2,0,0,0,13.7],
        'Prédiction N°2': [1,0,1,0,2,1,1,1,1,2,0,1,0,0,1,2,1,0,1,18.0]
    }


tab = pd.DataFrame.from_dict(data_pred, orient='index')
        # Définir les colonnes en utilisant la première ligne du DataFrame
tab.columns = tab.iloc[0]
        # Exclure la première ligne du DataFrame
tab = tab[1:]    

tab.head()


Variables,salaire_cadre_discretise,salaire_cadre_moyen_discretise,salaire_employe_discretise,salaire_travailleur_discretise,salaire_cadre_femme_discretise,salaire_cadre_moyen_femme_discretise,salaire_employe_femme_discretise,salaire_travailleur_femme_discretise,salaire_homme_discretise,salaire_cadre_homme_discretise,salaire_cadre_moyen_homme_discretise,salaire_employe_homme_discretise,salaire_travailleur_homme_discretise,salaire_18-25_discretise,salaire_+50_discretise,salaire_18-25_femme_discretise,salaire_+50_femme_discretise,salaire_18-25_homme_discretise,salaire_+50_homme_discretise,salaire
Prédiction N°1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,2,0,0,0,13.7
Prédiction N°2,1,0,1,0,2,1,1,1,1,2,0,1,0,0,1,2,1,0,1,18.0


In [12]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.5.1.


In [13]:
from platform import python_version

print(python_version())

3.11.5


In [14]:
import matplotlib
print('The scikit-learn version is {}.'.format(pandas.__version__))

NameError: name 'pandas' is not defined

In [ ]:

    # Charger le modèle et le mapping de la cible
    modele = charger_modele()
    target_mapping = charger_target_mapping()
    
    # Préparer les caractéristiques pour la prédiction
    caracteristiques = np.array([caracteristiques_entree])
    
    # Prévoir la classe avec le modèle
    prediction_encoded = modele.predict(caracteristiques)
    
    # Décoder la prédiction
    #prediction_decoded = target_mapping[prediction_encoded[0]]
    
    # Afficher la prédiction
    st.markdown(
        "{prediction_encoded}", 
        unsafe_allow_html=True